In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt

from collections import namedtuple
from extra_data import RunDirectory
from extra_data.read_machinery import find_proposal
from extra_redu.litfrm import make_litframe_finder, FrameSelection, SelType
from extra_redu import ReferenceDelay
from extra_redu.litfrm.draw.agipd import draw_cells

In [ ]:
RunId = namedtuple("RunId", ["propno", "runno"])
examples = [
    RunId(2899, 295),   #  0 - MID
    RunId(3246, 187),   #  1 - HED
    RunId(2844, 100),   #  2 - HED
    RunId(2834, 174),   #  3 - MID
    RunId(2834, 50),    #  4 - MID
    RunId(2671, 409),   #  5 - SPB
    RunId(2841, 300),   #  6 - MID
    RunId(900310, 92),  #  7 - MID (dev. ver. 0.4.0)
    RunId(900310, 93),  #  8 - MID (dev. ver. 0.4.0)
    RunId(3052, 59),    #  9 - MID (dev. ver. 0.4.0, DataSelector)
    RunId(3118, 18),    # 10 - MID (dev. ver. 0.4.0, DataSelector, RollingBunchPattern)
    RunId(3348, 87),    # 11 - MID (rolling bunch pattern, range step issue, two identical patterns in a row)
    RunId(4577, 86),    # 12 - SPB (range step issue, one last train with empty pattern)
    RunId(9037, 6),     # 13 - MID (time scan)
    RunId(900514, 93),  # 14 - HED
]

ex = examples[14]
print(ex)

propdir = find_proposal(f"p{ex.propno:06d}")
rundir = os.path.join(propdir, f"raw/r{ex.runno:04d}")

run = RunDirectory(rundir)
inst = rundir.split('/')[4]

In [ ]:
dev = make_litframe_finder(inst, run)

In [ ]:
tm0 = time.monotonic()
r1 = dev.process(True)
tid1 = r1.meta.trainId
tm1 = time.monotonic()

print(f"N trains: {tid1.size}, time: {tm1-tm0:.2f}s, train per sec: {tid1.size/(tm1-tm0):.2f}")

In [ ]:
tm0 = time.monotonic()
r2 = dev.read()
tid2 = r2.meta.trainId
tm1 = time.monotonic()

print(f"N trains: {tid2.size}, time: {tm1-tm0:.2f}s, train per sec: {tid2.size/(tm1-tm0):.2f}")

In [ ]:
tm0 = time.monotonic()
r3 = dev.read_or_process()
tid3 = r3.meta.trainId
tm1 = time.monotonic()

print(f"N trains: {tid3.size}, time: {tm1-tm0:.2f}s, train per sec: {tid3.size/(tm1-tm0):.2f}")

In [ ]:
tid, r = tid3, r3
sel = FrameSelection(r, guess_missed=True)

print("gray - unused, blue - dark, orange - lit")
print()
print(f"Proposal: {ex.propno}, run: {ex.runno}, source: {r.meta.litFrmDev}")
print()

for rec in sel.report():
    frmintf = ', '.join(
        [':'.join([str(n) for n in slc]) for slc in rec['litframe_slice']]
    )
    t0, tN, st = (rec['train_range'] + (1,))[:3]
    ntrain = max((int(tN) - int(t0)) // int(st), 1)
    trsintf = ':'.join([str(n) for n in rec['train_range']])
    print(" # trains                     Ntrn Nmis   Np  Nd  Nf lit frames")
    print(
        ("{pattern_no:2d} {trsintf:25s} {ntrain:5d} {nmissed_trains:4d} "
         "{npulse_exposed:4d} {ndataframe:3d} {nframe_total:3d} [{frmintf}]"
        ).format(frmintf=frmintf, ntrain=ntrain, trsintf=trsintf, **rec)
    )
    i = np.flatnonzero(rec['train_range'][0] == tid)[0]
    cflag = r.output.dataFramePattern[i]
    ncell = r.output.nFrame[i]
    draw_cells(cflag[:ncell])
    plt.show()
